# Ensemble Analysis and Multi-Model Evaluation with Monet Stats

This notebook demonstrates ensemble forecasting analysis and multi-model evaluation techniques using Monet Stats. We'll work with ensemble forecasts and explore various probabilistic verification methods.

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# For xarray support
import monet_stats as ms

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Load Temperature Dataset with Ensemble Forecasts

We'll use the synthetic temperature dataset that includes ensemble forecasts.

In [ ]:
# Load the example temperature dataset
temp_df = pd.read_csv('../data/temperature_obs_mod.csv')

# For ensemble analysis, we'll generate synthetic ensemble forecasts
# based on the existing observed and modeled data
obs_temps = temp_df['observed_temp'].values
mod_temps = temp_df['modeled_temp'].values

print("Dataset shape:", temp_df.shape)
print("\nFirst few rows:")
print(temp_df.head())

# Generate synthetic ensemble forecasts
n_members = 10 # Number of ensemble members
n_samples = len(obs_temps)

# Create ensemble forecasts with systematic differences and random variations
ensemble_forecasts = np.zeros((n_samples, n_members))
for i in range(n_members):
    # Each ensemble member has slightly different bias and random error
    member_bias = np.random.normal(0, 0.5)  # Random bias
    member_noise = np.random.normal(0, np.random.uniform(0.5, 1.5), n_samples)  # Random noise

    # Ensemble member forecast
    ensemble_forecasts[:, i] = mod_temps + member_bias + member_noise

print(f"\nGenerated ensemble forecasts with {n_members} members")
print(f"Ensemble shape: {ensemble_forecasts.shape}")
print(f"Ensemble mean bias: {np.mean(np.mean(ensemble_forecasts, axis=1) - obs_temps):.4f}")
print(f"Ensemble spread (mean std): {np.mean(np.std(ensemble_forecasts, axis=1)):.4f}")

## Ensemble Statistics

Calculate basic ensemble statistics and metrics.

In [ ]:
# Calculate ensemble statistics
ensemble_mean = np.mean(ensemble_forecasts, axis=1)
ensemble_std = np.std(ensemble_forecasts, axis=1)
ensemble_min = np.min(ensemble_forecasts, axis=1)
ensemble_max = np.max(ensemble_forecasts, axis=1)

print("Ensemble Statistics:")
print(f"Mean of ensemble means: {np.mean(ensemble_mean):.4f}")
print(f"Mean of ensemble std: {np.mean(ensemble_std):.4f}")
print(f"Overall ensemble range: {np.min(ensemble_min):.4f} to {np.max(ensemble_max):.4f}")

# Calculate metrics for ensemble mean vs observations
print("\nEnsemble Mean Performance:")
print(f"MAE (Ensemble Mean vs Obs): {ms.MAE(obs_temps, ensemble_mean):.4f}")
print(f"RMSE (Ensemble Mean vs Obs): {ms.RMSE(obs_temps, ensemble_mean):.4f}")
print(f"MBE (Ensemble Mean vs Obs): {ms.MBE(obs_temps, ensemble_mean):.4f}")
print(f"Correlation (Ensemble Mean vs Obs): {ms.pearson_correlation(obs_temps, ensemble_mean):.4f}")
print(f"R² (Ensemble Mean vs Obs): {ms.R2(obs_temps, ensemble_mean):.4f}")

# Calculate metrics for single model vs observations
print("\nSingle Model Performance:")
print(f"MAE (Model vs Obs): {ms.MAE(obs_temps, mod_temps):.4f}")
print(f"RMSE (Model vs Obs): {ms.RMSE(obs_temps, mod_temps):.4f}")
print(f"MBE (Model vs Obs): {ms.MBE(obs_temps, mod_temps):.4f}")
print(f"Correlation (Model vs Obs): {ms.pearson_correlation(obs_temps, mod_temps):.4f}")
print(f"R² (Model vs Obs): {ms.R2(obs_temps, mod_temps):.4f}")

# Compare ensemble vs single model performance
print("\nPerformance Comparison:")
print(f"Ensemble vs Single Model RMSE Improvement: {(ms.RMSE(obs_temps, mod_temps) - ms.RMSE(obs_temps, ensemble_mean)) / ms.RMSE(obs_temps, mod_temps) * 100:.2f}%")

## Probabilistic Verification Metrics

Calculate probabilistic verification metrics for ensemble forecasts.

In [ ]:
# Calculate probabilistic verification metrics
print("Probabilistic Verification Metrics:")

# Brier Score for different temperature thresholds
thresholds = [0, 10, 20, 30]  # Temperature thresholds in Celsius
for thresh in thresholds:
    # Binary observation (1 if temp > threshold, 0 otherwise)
    obs_binary = (obs_temps > thresh).astype(int)

    # Probability from ensemble (fraction of members above threshold)
    prob_forecast = np.mean(ensemble_forecasts > thresh, axis=1)

    # Calculate Brier Score
    bs = ms.BS(obs_binary, prob_forecast)
    print(f"Brier Score (T > {thresh}°C): {bs:.4f}")

# Continuous Ranked Probability Score (CRPS)
# Since we don't have the exact implementation in monet_stats, we'll use a simplified approach
# For demonstration purposes, we'll calculate a related metric
print("\nEnsemble Spread vs Error:")
abs_errors = np.abs(ensemble_mean - obs_temps)
spread_error_corr = ms.pearson_correlation(ensemble_std, abs_errors)
print(f"Correlation between ensemble spread and absolute error: {spread_error_corr:.4f}")

# Reliability analysis
def reliability_analysis(obs, prob_forecasts, n_bins=10):
    """Perform reliability analysis"""
    bin_edges = np.linspace(0, 1, n_bins + 1)
    observed_freq = []
    forecast_prob = []
    bin_counts = []

    for i in range(n_bins):
        bin_mask = (prob_forecasts >= bin_edges[i]) & (prob_forecasts < bin_edges[i+1])
        if i == n_bins - 1:  # Include upper bound for last bin
            bin_mask = (prob_forecasts >= bin_edges[i]) & (prob_forecasts <= bin_edges[i+1])

        if np.sum(bin_mask) > 0:
            observed_freq.append(np.mean(obs[bin_mask]))
            forecast_prob.append(np.mean(prob_forecasts[bin_mask]))
            bin_counts.append(np.sum(bin_mask))
        else:
            observed_freq.append(np.nan)
            forecast_prob.append(np.mean([bin_edges[i], bin_edges[i+1]]))
            bin_counts.append(0)

    return np.array(observed_freq), np.array(forecast_prob), np.array(bin_counts)

# Perform reliability analysis for one threshold
thresh = 20
obs_binary = (obs_temps > thresh).astype(int)
prob_forecast = np.mean(ensemble_forecasts > thresh, axis=1)

obs_freq, fore_prob, counts = reliability_analysis(obs_binary, prob_forecast)

print(f"\nReliability Analysis (T > {thresh}°C):")
for i in range(len(obs_freq)):
    if not np.isnan(obs_freq[i]):
        print(f"Forecast bin {i+1}: {fore_prob[i]:.2f} forecast, {obs_freq[i]:.2f} observed, {counts[i]} samples")
    else:
        print(f"Forecast bin {i+1}: {fore_prob[i]:.2f} forecast, no samples")

## Ensemble Verification Visualization

Create visualizations to understand ensemble forecast performance.

In [ ]:
# Create ensemble verification visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Ensemble vs observed scatter plot
axes[0, 0].scatter(obs_temps[:500], ensemble_mean[:500], alpha=0.5, s=20)
axes[0, 0].plot([obs_temps.min(), obs_temps.max()], [obs_temps.min(), obs_temps.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Observed Temperature (°C)')
axes[0, 0].set_ylabel('Ensemble Mean Temperature (°C)')
axes[0, 0].set_title(f'Ensemble Mean vs Observed (R² = {ms.R2(obs_temps, ensemble_mean):.3f})')
axes[0, 0].grid(True, alpha=0.3)

# Single model vs observed scatter plot
axes[0, 1].scatter(obs_temps[:500], mod_temps[:500], alpha=0.5, s=20)
axes[0, 1].plot([obs_temps.min(), obs_temps.max()], [obs_temps.min(), obs_temps.max()], 'r--', lw=2)
axes[0, 1].set_xlabel('Observed Temperature (°C)')
axes[0, 1].set_ylabel('Single Model Temperature (°C)')
axes[0, 1].set_title(f'Single Model vs Observed (R² = {ms.R2(obs_temps, mod_temps):.3f})')
axes[0, 1].grid(True, alpha=0.3)

# Ensemble spread vs absolute error
abs_errors = np.abs(ensemble_mean - obs_temps)
axes[0, 2].scatter(ensemble_std, abs_errors, alpha=0.5, s=20)
axes[0, 2].set_xlabel('Ensemble Spread (°C)')
axes[0, 2].set_ylabel('Absolute Error (°C)')
axes[0, 2].set_title(f'Spread vs Error (Corr = {ms.pearson_correlation(ensemble_std, abs_errors):.3f})')
axes[0, 2].grid(True, alpha=0.3)

# Time series with ensemble range
n_plot = min(100, len(obs_temps))
time_idx = range(n_plot)
axes[1, 0].plot(time_idx, obs_temps[:n_plot], label='Observed', linewidth=2)
axes[1, 0].plot(time_idx, ensemble_mean[:n_plot], label='Ensemble Mean', linewidth=2)
axes[1, 0].fill_between(time_idx, ensemble_min[:n_plot], ensemble_max[:n_plot], alpha=0.3, label='Ensemble Range')
axes[1, 0].set_xlabel('Time Index')
axes[1, 0].set_ylabel('Temperature (°C)')
axes[1, 0].set_title('Time Series with Ensemble Range')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Reliability diagram
axes[1, 1].plot(fore_prob, obs_freq, 'o-', linewidth=2, markersize=8, label='Actual')
axes[1, 1].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Perfect Reliability')
axes[1, 1].set_xlabel('Forecast Probability')
axes[1, 1].set_ylabel('Observed Frequency')
axes[1, 1].set_title('Reliability Diagram (T > 20°C)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xlim(0, 1)
axes[1, 1].set_ylim(0, 1)

# Rank histogram (TBD: Simplified version)
# Calculate ranks of observations relative to ensemble members
n_rank_samples = min(1000, len(obs_temps))
sample_indices = np.random.choice(len(obs_temps), n_rank_samples, replace=False)
sample_obs = obs_temps[sample_indices]
sample_ens = ensemble_forecasts[sample_indices, :]

# Calculate ranks
ranks = []
for i in range(len(sample_obs)):
    all_values = np.concatenate([[sample_obs[i]], sample_ens[i, :]])
    sorted_indices = np.argsort(all_values)
    obs_rank = np.where(sorted_indices == 0)[0][0] # 0 is the observation's position in sorted array
    ranks.append(obs_rank)

rank_counts, _ = np.histogram(ranks, bins=np.arange(0, n_members + 2))
axes[1, 2].bar(range(len(rank_counts)), rank_counts, edgecolor='black', alpha=0.7)
axes[1, 2].set_xlabel('Rank')
axes[1, 2].set_ylabel('Count')
axes[1, 2].set_title('Rank Histogram (Verification Rank)')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Multi-Model Ensemble Analysis

Simulate and analyze a multi-model ensemble.

In [ ]:
# Create synthetic multi-model ensemble
n_models = 5  # Number of different models in the multi-model ensemble
n_samples = len(obs_temps)

# Generate forecasts from different models
multi_model_forecasts = np.zeros((n_samples, n_models))
model_names = [f'Model_{i+1}' for i in range(n_models)]

for i in range(n_models):
    # Each model has different characteristics
    model_bias = np.random.normal(0, 1)  # Different bias for each model
    model_corr = np.random.uniform(0.6, 0.95)  # Different correlation
    model_noise = np.random.normal(0, np.random.uniform(0.5, 2.0), n_samples)  # Different noise level

    # Create correlated forecast with observations
    multi_model_forecasts[:, i] = (
        obs_temps * model_corr +  # Correlated part
        (1 - model_corr) * np.mean(obs_temps) +  # Climatological part
        model_bias +  # Systematic bias
        model_noise  # Random noise
    )

print("Multi-Model Ensemble Created:")
for i, model_name in enumerate(model_names):
    model_rmse = ms.RMSE(obs_temps, multi_model_forecasts[:, i])
    model_corr = ms.pearson_correlation(obs_temps, multi_model_forecasts[:, i])
    print(f"{model_name}: RMSE={model_rmse:.4f}, Correlation={model_corr:.4f}")

# Calculate multi-model ensemble statistics
mm_mean = np.mean(multi_model_forecasts, axis=1)
mm_std = np.std(multi_model_forecasts, axis=1)

print("\nMulti-Model Ensemble Performance:")
print(f"Mean RMSE: {ms.RMSE(obs_temps, mm_mean):.4f}")
print(f"Mean Correlation: {ms.pearson_correlation(obs_temps, mm_mean):.4f}")

# Compare single best model vs multi-model ensemble
model_rmses = [ms.RMSE(obs_temps, multi_model_forecasts[:, i]) for i in range(n_models)]
best_model_idx = np.argmin(model_rmses)
best_model_rmse = model_rmses[best_model_idx]

print(f"\nBest Single Model (Model_{best_model_idx+1}): RMSE={best_model_rmse:.4f}")
print(f"Multi-Model Ensemble: RMSE={ms.RMSE(obs_temps, mm_mean):.4f}")
print(f"Ensemble Improvement: {(best_model_rmse - ms.RMSE(obs_temps, mm_mean)) / best_model_rmse * 100:.2f}%")

# Calculate model weights based on performance
model_weights = 1.0 / np.array(model_rmses)  # Inverse of RMSE as weight
model_weights = model_weights / np.sum(model_weights)  # Normalize

# Calculate weighted ensemble mean
weighted_mean = np.zeros(n_samples)
for i in range(n_models):
    weighted_mean += model_weights[i] * multi_model_forecasts[:, i]

print(f"\nWeighted Multi-Model Ensemble: RMSE={ms.RMSE(obs_temps, weighted_mean):.4f}")
print(f"Weights: {model_weights}")

## Ensemble Ranking and Skill Assessment

Rank ensemble members and assess their individual skills.

In [ ]:
# Rank ensemble members by performance
member_metrics = []
for i in range(n_members):
    member_forecasts = ensemble_forecasts[:, i]
    member_metrics.append({
        'member': f'Member_{i+1}',
        'RMSE': ms.RMSE(obs_temps, member_forecasts),
        'MAE': ms.MAE(obs_temps, member_forecasts),
        'Correlation': ms.pearson_correlation(obs_temps, member_forecasts),
        'MBE': ms.MBE(obs_temps, member_forecasts)
    })

# Convert to DataFrame and sort by RMSE
member_df = pd.DataFrame(member_metrics)
member_df = member_df.sort_values('RMSE')
member_df['Rank'] = range(1, len(member_df) + 1)

print("Ensemble Members Ranked by RMSE:")
print(member_df[['Rank', 'member', 'RMSE', 'Correlation', 'MBE']].round(4))

# Calculate ensemble skill scores
print("\nEnsemble Skill Scores:")

# Calculate RMSE of ensemble mean vs climatology reference
climatology = np.mean(obs_temps)
rmse_ensemble = ms.RMSE(obs_temps, ensemble_mean)
rmse_clim = ms.RMSE(obs_temps, np.full_like(obs_temps, climatology))

ss_rmse = ms.SS(mse_model=rmse_ensemble**2, mse_reference=rmse_clim**2)
print(f"RMSE Skill Score: {ss_rmse:.4f}")

# Nash-Sutcliffe Efficiency
nse = ms.NSE(obs_temps, ensemble_mean)
print(f"NSE: {nse:.4f}")

# Calculate ensemble spread-skill relationship
abs_errors = np.abs(ensemble_mean - obs_temps)
spread_skill_corr = ms.pearson_correlation(ensemble_std, abs_errors)
print(f"Spread-Skill Correlation: {spread_skill_corr:.4f}")

# Calculate ensemble reliability metrics
reliability_metrics = []
for thresh in [10, 20]:
    obs_binary = (obs_temps > thresh).astype(int)
    prob_forecast = np.mean(ensemble_forecasts > thresh, axis=1)
    bs = ms.BS(obs_binary, prob_forecast)
    reliability_metrics.append({'threshold': f'T>{thresh}°C', 'Brier_Score': bs})

print("\nReliability Metrics:")
for rm in reliability_metrics:
    print(f"{rm['threshold']}: Brier Score = {rm['Brier_Score']:.4f}")

## Ensemble Forecast Evaluation Dashboard

Create a comprehensive dashboard for ensemble evaluation.

In [ ]:
# Create ensemble evaluation dashboard
fig, axes = plt.subplots(3, 3, figsize=(18, 15))

# Ensemble member performance comparison
rmse_values = [ms.RMSE(obs_temps, ensemble_forecasts[:, i]) for i in range(n_members)]
axes[0, 0].bar(range(1, n_members + 1), rmse_values)
axes[0, 0].set_xlabel('Ensemble Member')
axes[0, 0].set_ylabel('RMSE')
axes[0, 0].set_title('RMSE by Ensemble Member')
axes[0, 0].grid(True, alpha=0.3)

# Multi-model comparison
mm_rmses = [ms.RMSE(obs_temps, multi_model_forecasts[:, i]) for i in range(n_models)]
all_rmses = rmse_values + mm_rmses
labels = [f'E{i+1}' for i in range(n_members)] + [f'MM{i+1}' for i in range(n_models)]
colors = ['blue'] * n_members + ['red'] * n_models

axes[0, 1].bar(range(len(all_rmses)), all_rmses, color=colors)
axes[0, 1].set_xlabel('Model/Member')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].set_title('RMSE Comparison (Ensemble vs Multi-Model)')
axes[0, 1].set_xticks(range(len(labels)))
axes[0, 1].set_xticklabels(labels, rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Ensemble spread histogram
axes[0, 2].hist(ensemble_std, bins=30, edgecolor='black', alpha=0.7)
axes[0, 2].set_xlabel('Ensemble Spread')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title(f'Ensemble Spread Distribution (Mean: {np.mean(ensemble_std):.3f})')
axes[0, 2].grid(True, alpha=0.3)

# Probability vs observed for threshold
thresh = 15
obs_binary = (obs_temps > thresh).astype(int)
prob_forecast = np.mean(ensemble_forecasts > thresh, axis=1)

axes[1, 0].scatter(prob_forecast, obs_binary, alpha=0.5, s=20)
axes[1, 0].set_xlabel('Forecast Probability (T > 15°C)')
axes[1, 0].set_ylabel('Observed (1) or Not (0)')
axes[1, 0].set_title('Probability vs Observed (T > 15°C)')
axes[1, 0].grid(True, alpha=0.3)

# Ensemble forecast uncertainty vs performance
axes[1, 1].scatter(ensemble_std, abs_errors, alpha=0.5, s=20)
axes[1, 1].set_xlabel('Ensemble Uncertainty (Std)')
axes[1, 1].set_ylabel('Absolute Error')
axes[1, 1].set_title(f'Uncertainty vs Performance (Corr: {spread_skill_corr:.3f})')
axes[1, 1].grid(True, alpha=0.3)

# ROC curve for one threshold
from sklearn.metrics import auc, roc_curve

fpr, tpr, _ = roc_curve(obs_binary, prob_forecast)
roc_auc = auc(fpr, tpr)

axes[1, 2].plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.3f})')
axes[1, 2].plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random')
axes[1, 2].set_xlabel('False Positive Rate')
axes[1, 2].set_ylabel('True Positive Rate')
axes[1, 2].set_title('ROC Curve (T > 15°C)')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

# Ensemble forecast distribution for a single time point
time_idx = 50
axes[2, 0].hist(ensemble_forecasts[time_idx, :], bins=10, edgecolor='black', alpha=0.7)
axes[2, 0].axvline(obs_temps[time_idx], color='red', linewidth=2, label=f'Observed: {obs_temps[time_idx]:.2f}')
axes[2, 0].axvline(ensemble_mean[time_idx], color='blue', linewidth=2, label=f'Ensemble Mean: {ensemble_mean[time_idx]:.2f}')
axes[2, 0].set_xlabel('Temperature (°C)')
axes[2, 0].set_ylabel('Count')
axes[2, 0].set_title(f'Ensemble Distribution at Time {time_idx}')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

# Ranked probability score components
axes[2, 1].plot(range(1, n_members + 1), rmse_values, 'o-', linewidth=2, markersize=8)
axes[2, 1].set_xlabel('Ensemble Member (Ranked)')
axes[2, 1].set_ylabel('RMSE')
axes[2, 1].set_title('Ensemble Member Performance')
axes[2, 1].grid(True, alpha=0.3)

# Multi-model performance
axes[2, 2].bar(range(1, n_models + 1), mm_rmses, alpha=0.7)
axes[2, 2].set_xlabel('Multi-Model')
axes[2, 2].set_ylabel('RMSE')
axes[2, 2].set_title('Multi-Model Performance')
axes[2, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated ensemble analysis and multi-model evaluation techniques:

1. **Ensemble Statistics**: Mean, spread, min/max, and performance metrics
2. **Probabilistic Verification**: Brier Score, reliability analysis, rank histograms
3. **Multi-Model Ensembles**: Combining forecasts from different models
4. **Ensemble Ranking**: Assessing individual member performance
5. **Skill Assessment**: Ensemble skill scores and spread-skill relationships
6. **Comprehensive Dashboard**: Multi-faceted ensemble evaluation

These methods provide a complete framework for evaluating ensemble and multi-model forecasting systems in atmospheric sciences.